In [7]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

from tqdm import tqdm

import sys

### Loading Data

In [8]:
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=sys.maxsize)

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, data_path, labels_path):
        self.labels = torch.from_numpy(np.fromfile("labels", sep=" ")[1::2]).float()
        self.filenames = np.fromfile("labels", sep=" ")[0::2].astype(int)
        self.data_path = data_path

        self.data = {}
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        if i in self.data:
            vector = self.data[i]
        else:
            vector = np.fromfile(f"{self.data_path}/{self.filenames[i]}", sep=" ")[2::3]
            self.data[i] = vector
        return torch.from_numpy(vector).float(), torch.reshape(self.labels[i], (1,))

In [10]:
training_data = CustomImageDataset("Training", "labels")

In [11]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)

In [12]:
len(training_data)

2086

In [13]:
iterable = iter(train_dataloader)

In [14]:
next(iterable)[0].shape

torch.Size([8, 18422])

### Neural Network

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(122 * 151, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.linear_relu_stack(x)

In [16]:
test = torch.from_numpy(np.fromfile("Training/0", sep=" ")[2::3]).float()
# test, test_label = next(iterable)
test.shape

torch.Size([18422])

In [17]:
model = NeuralNetwork()

In [18]:
logits = model(test)
logits

tensor([0.5041], grad_fn=<SigmoidBackward0>)

In [19]:
pred_probab = nn.Softmax(dim=0)(logits)
pred_probab
answer = pred_probab.argmax(0)
answer

tensor(0)

In [20]:
answer.shape

torch.Size([])

### Training

In [21]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backward propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [22]:
learning_rate = 1e-3
batch_size = 8
epochs = 10

In [23]:
loss_fn = nn.BCELoss()

In [24]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
for t in range(epochs):
    print(f"Epoch {t + 1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)

Epoch 1
----------------------------------
loss: 0.699516 [    8/ 2086]
loss: 0.670389 [  808/ 2086]
loss: 0.640140 [ 1608/ 2086]
Epoch 2
----------------------------------
loss: 0.624408 [    8/ 2086]
loss: 0.596633 [  808/ 2086]
loss: 0.572746 [ 1608/ 2086]
Epoch 3
----------------------------------
loss: 0.557680 [    8/ 2086]
loss: 0.533692 [  808/ 2086]
loss: 0.512846 [ 1608/ 2086]
Epoch 4
----------------------------------
loss: 0.552256 [    8/ 2086]
loss: 0.539489 [  808/ 2086]
loss: 0.458695 [ 1608/ 2086]
Epoch 5
----------------------------------
loss: 0.446540 [    8/ 2086]
loss: 0.506546 [  808/ 2086]
loss: 0.411435 [ 1608/ 2086]
Epoch 6
----------------------------------
loss: 0.399804 [    8/ 2086]
loss: 0.472861 [  808/ 2086]
loss: 0.564189 [ 1608/ 2086]
Epoch 7
----------------------------------
loss: 0.458768 [    8/ 2086]
loss: 0.342827 [  808/ 2086]
loss: 0.439515 [ 1608/ 2086]
Epoch 8
----------------------------------
loss: 0.320420 [    8/ 2086]
loss: 0.306002 [  

In [37]:
for i in range(5000, 6000):
    test_arr = torch.from_numpy(np.fromfile(f"Training/{i}", sep=" ")[2::3]).float()
    print(model(test_arr))

tensor([0.2041], grad_fn=<SigmoidBackward0>)
tensor([0.2049], grad_fn=<SigmoidBackward0>)
tensor([0.2149], grad_fn=<SigmoidBackward0>)
tensor([0.2007], grad_fn=<SigmoidBackward0>)
tensor([0.2015], grad_fn=<SigmoidBackward0>)
tensor([0.1990], grad_fn=<SigmoidBackward0>)
tensor([0.2016], grad_fn=<SigmoidBackward0>)
tensor([0.1928], grad_fn=<SigmoidBackward0>)
tensor([0.2013], grad_fn=<SigmoidBackward0>)
tensor([0.2037], grad_fn=<SigmoidBackward0>)
tensor([0.2127], grad_fn=<SigmoidBackward0>)
tensor([0.2154], grad_fn=<SigmoidBackward0>)
tensor([0.2159], grad_fn=<SigmoidBackward0>)
tensor([0.1979], grad_fn=<SigmoidBackward0>)
tensor([0.1975], grad_fn=<SigmoidBackward0>)
tensor([0.1995], grad_fn=<SigmoidBackward0>)
tensor([0.2002], grad_fn=<SigmoidBackward0>)
tensor([0.1976], grad_fn=<SigmoidBackward0>)
tensor([0.1992], grad_fn=<SigmoidBackward0>)
tensor([0.2050], grad_fn=<SigmoidBackward0>)
tensor([0.2002], grad_fn=<SigmoidBackward0>)
tensor([0.2003], grad_fn=<SigmoidBackward0>)
tensor([0.